In [21]:
from __future__ import print_function

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import sklearn as slearn



from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

print('Python: {}'.format(sys.version))
print('Seaborn {}'.format(sns.__version__))
print('sklearn: {}'.format(slearn.__version__))

Python: 3.8.1 (tags/v3.8.1:1b293b6, Dec 18 2019, 22:39:24) [MSC v.1916 32 bit (Intel)]
Seaborn 0.10.0
sklearn: 0.22.2.post1


# Reading Data From CSV file 

In [22]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [23]:
#confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')

In [24]:
death_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,11,14,14,15,15,18,18,21,23,25
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,21,22,22,23,23,23,23,23,24,25
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,173,193,205,235,256,275,293,313,326,336
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,21,22,23,25,26,26,29,29,31,33
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [25]:
#confirmed_df.head()

In [26]:
#data cleaning - rename columns
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

#death_df.columns

In [27]:
#confirmed_df.columns

# Renaming column name 

In [28]:
# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})

In [29]:
#death_df.columns

In [30]:
#country_df.columns

In [31]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [32]:
import plotly.express as px

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show()

In [33]:
interact(bubble_chart, n=20);

interactive(children=(IntSlider(value=20, description='n', max=60, min=-20), Output()), _dom_classes=('widget-…

In [34]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [35]:
def highlight_col(x):
    r = 'background-color: tomato'
    p = 'background-color: gold'
    g = 'background-color: yellowgreen'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = p
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    return df1

In [36]:
def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10');

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

In [37]:
px.bar(
        sorted_country_df.head(10),
        x = "country",
        y = "deaths",
        title= "Top 10 worst affected countries", # the axis names
        color_discrete_sequence=["crimson"], 
        height=500,
        width=800
    )

In [38]:
import plotly.graph_objects as go

def plot_cases_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue','red']
    df_list = [confirmed_df, death_df]
    line_size = [4,5]
    
    fig = go.Figure()

    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            
            x_data = np.array(list(df.iloc[:, 5:].columns))            
            y_data = np.sum(np.asarray(df.iloc[ : , 5:]), axis=0 )
            
        else:
            x_data = np.array(list(df.iloc[:, 5:].columns))            
            y_data = np.sum(np.asarray(df[df['country'] == country].loc[: , 5:]) , axis = 0)
        
        fig.add_trace( go.Scatter (x=x_data, y=y_data, mode='lines+markers', 
                                   name = labels[i], line = dict(color = colors[i], width = line_size[i]), 
                                  connectgaps = True, text = "Total" + str(labels[i]) + str(y_data[-1]) 
                                  ))
    fig.show()        

In [39]:
#plot_cases_country('World')

interact(plot_cases_country, country='World');

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…